<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 4 : Bases de données relationnelles</h1>

Le modèle relationnel est un modèle mathématique permettant de raisonner sur des données tabulées.  
Il est mis en œuvre par un logiciel particulier, le **système de gestion de bases de données** (SGBD en abrégé).  
Un **SGBD relationnel** est un SGBD utilisant le **modèle relationnel** pour la représentation des données.  
L'écrasante majorité des SGBD relationnels utilisent le langage SQL (**Structured Query Language**, langage de requête structuré).  
Ce dernier permet d'envoyer des ordres au SGDB.  
Les ordres peuvent être de deux natures.
* Les mises à jour permettent la création de relations, l'ajout d'entité dans ces dernières, leur modification et leur suppression. 
* Les requêtes permettent de récupérer les données répondant à des critères particuliers.


## SQL: un langage de définition de données
Directement inspiré du modèle relationnel introduit par E. Codd, le langage SQL permet la définition de relations ou tables dans une base de données relationnelle.  
Ce langage est standardisé par l'ISO, sous la référence [ISO/IEC 9075](https://www.iso.org/fr/standard/63555.html). La dernière version du standard date de 2016.  
Le langage SQL permet de créer des tables en spécifiant leur nom, leurs attributs, les types de ces derniers et les contraintes associées à la table.

Reprenons l'exemple de la médiathèque :

<div style="text-align: center">
   <img src="Images/Base2.png" alt="Base de données">
</div>

Pour créer les tables correspondant à la modélisation finale de la médiathèque, on peut saisir les ordres suivants:

```sql
CREATE TABLE usager (
    nom VARCHAR(90), 
    prenom VARCHAR(90), 
    adresse VARCHAR(300), 
    cp VARCHAR(5), 
    ville VARCHAR(60), 
    email VARCHAR(60), 
    code_barre CHAR(15) PRIMARY KEY
);
                     
CREATE TABLE livre (
    titre VARCHAR(300), 
    editeur VARCHAR(90), 
    annee INT, 
    isbn CHAR(14) PRIMARY KEY
);
                    
CREATE TABLE auteur (
    a_id INT PRIMARY KEY, 
    nom VARCHAR(90), 
    prenom VARCHAR(90)
);
                     
CREATE TABLE auteur_de (
    a_id INT REFERENCES auteur(a_id), 
    isbn CHAR(14) REFERENCES livre(isbn),
    PRIMARY KEY (a_id, isbn)
);
                        
CREATE TABLE emprunt (
    code_barre CHAR(15) REFERENCES usager(code_barre),  
    isbn CHAR(14) PRIMARY KEY REFERENCES livre(isbn), 
    retour DATE
);
```

L'interaction avec un SGBD se fait par l'envoi d'une suite d'ordres SQL.  
* Un ordre peut s'étendre sur plusieurs lignes. 
* Les blancs et l'indentation ne sont pas significatifs mais améliorent la lisibilité dans certains cas. 
* Un ordre se termine par un `;`.  

Les cinq ordres donnés ci-dessus créent les cinq tables composant notre base de données.  
* Une première remarque de syntaxe est que SQL est insensible à la casse.  
On aurait ainsi pu écrire `create table livre ...` ou encore `cReATE TablE lIVrE ... `.  
Nous faisons le choix d'utiliser des capitales pour les mots clés du langage SQL (`CREATE`, `TABLE`, `PRIMARY`, etc.) et des minuscules pour les noms d'attributs (`a_id`, `prenom`, etc.) et de tables.  
Ces derniers ne pouvant pas contenir d'espace, nous utilisons le caractère `_` comme séparateur de mots.  
Enfin, nous utilisons la convention, généralement considérée comme une bonne pratique, d'utiliser des noms de tables au singulier.  
* Une autre remarque de syntaxe est que le langage SQL peut sembler *verbeux*, à l'inverse d'un langage de programmation comme Python.  
Les ordres ressemblent à du langage naturel (en anglais).  
La notation est aussi semblable aux schémas du modèle relationnel.  

La syntaxe générale d'un ordre [`CREATE TABLE`](https://sqlite.org/lang_createtable.html) est la suivante:

    CREATE TABLE nom_table (
        att_1 dom_1 contr_1?,
        ... ,
        att_n dom_n contr_n?,
        contr_glob_1,
        ... ,
        contr_glob_n
    ) ;
                            
On donne, à la suite des mots clés `CREATE TABLE`, un nom de table, suivi de la liste des définitions d'attributs entre parenthèses et séparées par des virgules.  
Une définition d'attribut consiste en un nom d'attribut, un type d'attribut (ou domaine) qui sont tous les deux obligatoires et optionnellement des contraintes sur cet attribut.  
Si des contraintes portent sur plusieurs attributs à la fois (par exemple pour spécifier que plusieurs attributs forment une clé primaire) on peut placer ces contraintes en fin de liste, avant la parenthèse fermante.  

#### Remarque 
Le système lèvera une erreur si la table existe déjà.  
Il est possible d'utiliser l'opérateur [`EXISTS`](https://www.sqlite.org/lang_expr.html#the_exists_operator) pour vérifier l'existence de la table avant d'exécuter l'ordre : 

```sql
CREATE TABLE IF NOT EXISTS usager (
    nom VARCHAR(90), 
    prenom VARCHAR(90), 
    adresse VARCHAR(300), 
    cp VARCHAR(5), 
    ville VARCHAR(60), 
    email VARCHAR(60), 
    code_barre CHAR(15) PRIMARY KEY
);
```

#### Vocabulaire
Strictement parlant, une relation du modèle relationnel et une table SQL ne sont pas des concepts équivalents.  
En particulier, une table peut contenir des **doublons**.  
En effet, il n'est pas obligatoire de spécifier une clé primaire lors de la création d'une table SQL.  
Sans clé primaire, une table peut contenir plusieurs copies du même $n$-uplet, sans que cela pose problème, chose qui n'est, a priori, pas autorisé pour les ensembles du modèle relationnel (car justement, ce sont des ensembles).  
Nous utiliserons les termes **tables** et **relations** indistinctement.  
De façon analogue, en SQL les **attributs** d'une relation sont appelés des **colonnes** et les **entités** des **lignes**.

## Types de données en SQL
Les domaines abstraits du modèle relationnel correspondent à des [types de données](https://sqlite.org/datatype3.html) du langage SQL. 

### Types numériques
Le standard SQL définit plusieurs types numériques.  
Ces derniers sont soit des types numériques exacts, soit des types numériques approchés. 

| nom du type        | exact/approché | description                                             |
|:------------------|:--------------|:-------------------------------------------------------|
| `SMALLINT`         | exact          | entier 16 bits signé                                    |
| `INTEGER`          | exact          | entier 32 bits signé                                    |
| `INT`              | exact          | alias pour `INTEGER`                                    |
| `BIGINT`           | exact          | entier 64 bits signé                                    |
| `DECIMAL(t, f)`    | exact          | décimal signé de *t* chiffres dont *f* après la virgule |
| `REAL`             | approché       | flottant 32 bits                                        |
| `DOUBLE PRECISION` | approché       | flottant 64 bits                                        |

La plupart de ces types représentent fidèlement les entiers ou flottants *machine* manipulables directement par le processeur.  
Une exception notable est le type `DECIMAL(t, f)` qui permet de représenter de manière exacte un nombre à virgule d'une taille donnée.  
Ce type est particulièrement important, car il permet, par exemple, de représenter des sommes d'argent sans erreurs d'arrondis.  
Ainsi, le type `DECIMAL(5, 2)` permet de stocker des valeurs décimales de 5 chiffres, dont deux après la virgule, soit des valeurs entre -999,99 et 999, 99.  
Une utilisation judicieuse de ce type permettra de réaliser une contrainte de domaine sur un attribut numérique.  
Le standard ne supporte que les nombres en base 10, sans autoriser des notations hexadécimales ou octales comme dans les autres langages.

### Types textes
Le standard SQL définit plusieurs types permettant de stocker des chaînes de caractères.  
Malheureusement, ces derniers sont supportés de manière inégale dans les divers SGBD.  

| nom du type  | description                                                                                 |
|:------------|:-------------------------------------------------------------------------------------------|
| `CHAR(n)`    | Chaîne d'exactement *n* caractères. Les caractères manquant sont complétés par des espaces. |
| `VARCHAR(n)` | Chaîne d'au plus *n* caractères.                                                            |
| `TEXT`       | chaîne de taille quelconque.                                                                |

* Le type `CHAR(n)` permet de définir des chaînes de caractères de taille exactement *n*.  
La taille maximale acceptée pour *n* dépend des différents systèmes, mais ils supportent tous au moins 8000 caractères. La taille minimale est 1.  
Ce type est approprié lorsque l'on veut stocker des chaînes de taille fixe et connue, par exemple les ISBN de nos livres qui font exactement 14 caractères, de la forme `xxx-xxxxxxxxxx` (trois chiffres, un tiret, dix chiffres).  
Attention, si l'on stocke une chaîne de taille inférieure à *n*, cette dernière est complétée par la droite avec des espaces.  
Ainsi, la chaîne `'hello'` stockée dans une colonne de type `CHAR(10)` sera convertie en `'hello␣␣␣␣␣'` (où le caractère `␣` représente un espace).  

* Le type `VARCHAR(n)` permet, quant à lui, de définir des chaînes de taille au plus *n*.  
La valeur de *n* suit les mêmes règles que pour le type `CHAR`. 

* Enfin, le type `TEXT` permet de stocker des chaînes de caractères de taille variable, sans fixer de taille maximale a priori.  
En pratique, il est équivalent à `VARCHAR(n)` pour la plus grande valeur de *n* supportée par le système. 

Les chaînes de caractères littérales sont délimitées par des guillemets simples `'`.  
Le caractère guillemet peut être échappé en le doublant.  
Par exemple, la chaîne constante `'c'est moi'`s'écrira `'c''est moi'`.  
Les autres caractères n'ont pas besoin d'être échappés.  
Une chaîne peut en particulier contenir un retour chariot (et donc être écrite sur plusieurs lignes).

### Type booléen
Le type `BOOLEAN` est inégalement supporté par les différents systèmes, qu'ils soient commerciaux ou libres.  
Cela est dû au fait que le standard SQL laisse ce type comme optionnel.  
Les SGBD sont donc libres de ne pas le proposer.  
Une alternative possible est d'utiliser `CHAR(1)` et de se servir de deux caractères distincts (par exemple `'T'` et `'F'`) pour représenter des booléens.  
Une autre alternative est d'utiliser un type numérique exact et de considérer la valeur `0` comme fausse et les autres valeurs comme vraies.


### Type des dates, durées et instants
À première vue anodine, la gestion des dates et du temps est un problème excessivement complexe, source de
nombreux bugs.  
Le standard SQL propose donc de nombreux types temporels permettant de représenter des dates, des heures et des durées. 

| nom du type | description                                                  |
|:-----------|:------------------------------------------------------------|
| `DATE`      | Une date au format `'AAAA-MM-JJ'`                            |
| `TIME`      | Une heure au format `'hh:mm:ss'`                             |
| `TIMESTAMP` | Un instant (date et heure) au format `'AAAA-MM-JJ hh:mm:ss'` |

Les valeurs de ces types s'écrivent comme de simples chaînes de caractères.  
Une fonctionnalité intéressante est la possibilité d'utiliser l'addition pour ajouter des jours à une valeur de type `DATE`.  
Si `d` est une expression de type `DATE`, alors `d + 10` représente la date 10 jours après `d`.  
Cette opération produit une valeur de type `DATE` et donc prend correctement en compte les changements de mois, d'années et les années bissextiles. 

Nous ne détaillons pas plus ces types de données très complexes (qui savent par exemple prendre en compte les fuseaux horaires).

### Valeur NULL
Une valeur notée `NULL` existe en SQL.  
Elle représente une absence de valeur et peut donc être utilisée à la place de n'importe quelle autre valeur, quel que soit le type attendu.  
Son utilisation est similaire à la constante `None` du langage Python, mais son comportement est complexe et peut être source d'erreurs.  
Il est déconseillé de l'utiliser dans le cadre d'une initiation aux bases de données.  
En particulier, SQL interdit l'utilisation de `NULL` comme valeur pour une clé primaire.  
En revanche, elle est autorisée pour les clés étrangères.  
La valeur `NULL` permet donc de violer la contrainte de référence.  
La seule chose que l'on fera donc avec des attributs potentiellement `NULL` est de les tester au moyen des expression `e IS NULL` ou `e IS NOT NULL`.  
Attention cependant, le test `e = NULL` ne produit pas le résultat booléen comme on s'y attendrait mais renvoie toujours `NULL` quelle que soit la valeur de `e`.

## Spécification des contraintes d'intégrité
Les contraintes d'intégrité jouant un rôle fondamental dans le modèle relationnel, il est naturel de pouvoir les spécifier en SQL. 

### Clé primaire
Les mots clés [`PRIMARY KEY`](https://sqlite.org/lang_createtable.html#the_primary_key) permettent d'indiquer qu'un attribut est une clé primaire.  
Voici un exemple:

```sql
CREATE TABLE personne (
    id INT PRIMARY KEY, 
    nom VARCHAR(99), 
    prenom VARCHAR(99)
);
```

Si l'on souhaite utiliser plusieurs attributs comme clé primaire, on peut spécifier la contrainte après les attributs:

```sql
CREATE TABLE point (
    x INT, 
    y INT, 
    couleur VARCHAR(30), 
    PRIMARY KEY (x, y)
);
```

### Clé étrangère
Un attribut peut être qualifié de [clé étrangère](https://sqlite.org/foreignkeys.html) en utilisant le mot clé `REFERENCES` suivi de la table où se trouve la clé primaire et de son nom.

```sql
CREATE TABLE employe (
    emp INT REFERENCES personne(id), 
    dept VARCHAR(90), 
    suph INT REFERENCES personne(id)
);
```

La table `employe` associe un employé (attribut `emp`), qui est une personne, à son département dans l'entreprise (ressources humaines, support informatique, comptabilité, etc.) et à son supérieur hiérarchique (attribut `suph`), qui est aussi une personne.  
Ce lien est matérialisé par le fait que `emp` et `suph` sont des clés étrangères. 

Il est à noter que la plupart des SGBD ne supportent pas l'utilisation de clés étrangères composites.  L'utilité des clés primaires composites se trouve donc amoindrie en pratique.

### Unicité, non nullité
Il peut être intéressant de spécifier qu'un groupe d'attributs est unique, sans pour autant en faire une clé primaire. Cette information permet au SGBD plus de vérifications sur les données (cohérence des données) et parfois de traiter ces dernières de façon plus efficace.  
Cela peut être spécifié au moyen du mot clé [`UNIQUE`](https://sqlite.org/lang_createtable.html#unique_constraints).  
Une autre bonne pratique consiste à déclarer qu'un attribut ne peut pas être `NULL`. Cette valeur spéciale ne pourra donc jamais être utilisée pour remplir des valeurs de la colonne correspondante. Cela peut être fait au
moyen du mot clé [`NOT NULL`](https://sqlite.org/lang_createtable.html#not_null_constraints).  
Notons que `PRIMARY KEY` implique obligatoirement `NOT NULL`. 

En reprenant l'exemple des utilisateurs de la bibliothèque, on pourrait raffiner notre définition de table de la manière suivante :

```sql
CREATE TABLE usager (
    nom VARCHAR(90) NOT NULL, 
    prenom VARCHAR(90) NOT NULL, 
    adresse VARCHAR(300) NOT NULL, 
    cp VARCHAR(5) NOT NULL, 
    ville VARCHAR(60) NOT NULL, 
    email VARCHAR(60) NOT NULL UNIQUE, 
    code_barre CHAR(15) PRIMARY KEY
);
```

On spécifie de cette façon qu'aucun des attributs n'est optionnel et de plus que `email` doit être `UNIQUE` dans la table (même s'il n'est pas une clé primaire).

### Contraintes utilisateur
Il est possible de spécifier des contraintes arbitraires sur les attributs d'une même ligne au moyen du mot clé [`CHECK`](https://sqlite.org/lang_createtable.html#check_constraints), suivi d'une formule booléenne.  
Cette contrainte est placée obligatoirement en fin de déclaration avant la parenthèse fermante (et non pas au niveau d'un attribut). 

```sql
CREATE TABLE produit (
    id INT PRIMARY KEY, 
    nom VARCHAR(100) NOT NULL, 
    quantite INT NOT NULL, 
    prix DECIMAL(10,2) NOT NULL, 
    CHECK (quantite >= 0 AND prix >= 0)
);
```

Nous définissons ici une table des produits vendus dans un magasin.  
Ces derniers ont un identifiant (qui est la clé primaire), un nom, une quantité et un prix.  
Ceux-ci ne peuvent jamais être négatifs (ce qui n'est pas exprimable uniquement au moyen des types `INT` ou `DECIMAL`).  
On ajoute donc une contrainte `CHECK`.

## Suppression de tables
Une fois qu'une table est créée, il n'est pas possible d'en créer une autre avec le même nom.  
Si on souhaite recréer la table, par exemple avec un schéma différent, il faut d'abord supprimer celle portant le même nom.  
C'est le but de l'instruction [`DROP TABLE`](https://sqlite.org/lang_droptable.html):

```sql
DROP TABLE auteur_de;
```
Cette dernière supprime la table et donc toutes les données qui y sont stockées.  

#### Remarque
Un point important est que, **selon le type de SGBD utilisé** (ce n'est pas le cas de SQLite), il n'est parfois pas possible de supprimer une table si elle sert de référence pour une clé étrangère d'une autre table, car cela violerait une contrainte de référence :

```sql
DROP TABLE usager;
```
```
ERROR: cannot drop table usager because other objects depend on it
DETAIL: constraint emprunt_code_barre_fkey on table emprunt depends on table usager
```

Le système indique que la table `usager` est mentionnée par une contrainte (ici celle de la table `emprunt`).  
Le système refuse donc de supprimer la table et renvoie une erreur.  
Il convient donc, par mesure de précaution, de supprimer les tables dans le bon ordre, c'est-à-dire d'abord les tables contenant les clés étrangères, avant les tables contenant les clés primaires référencées.

```sql
DROP TABLE emprunt;
DROP TABLE auteur_de;
DROP TABLE auteur;
DROP TABLE livre;
DROP TABLE usager;
```

Certains SGBD permettent de spécifier que la suppression d'une table doit détruire automatiquement toutes les tables qui dépendent d'elle.  
Cette fonctionnalité est commode mais dangereuse et de toute façon inégalement supportée.  
Il est donc conseillé de spécifier explicitement les commandes de suppression, dans l'ordre adéquat.

#### Remarque 
Le système lèvera une erreur si la table n'existe pas.  
Il est possible d'utiliser l'opérateur [`EXISTS`](https://www.sqlite.org/lang_expr.html#the_exists_operator) pour vérifier l'existence de la table avant d'exécuter l'ordre : 

```sql
DROP TABLE IF EXISTS usager;
```

### Le standard SQL en pratique
Bien que très volumineux (le standard [ISO/IEC 9075](https://www.iso.org/standard/63555.html) qui définit la norme SQL se décompose en 14 parties distinctes et fait plus de 5000 pages), le standard SQL comporte beaucoup de parties optionnelles.  
En pratique, même si la plupart des SGBD commerciaux et libres en implémentent une partie, leur support est inégal.  
Il n'est donc pas rare de devoir écrire du code SQL *propriétaire*, c'est-à-dire,utilisant des extensions propres à SGBD particulier, dès que l'on souhaite faire des traitements un tant soit peu complexes.  
Cette pratique nuit malheureusement à la portabilité.

## Insertion dans une table
Nous pouvons enfin aborder l'insertion de nouvelles valeurs dans une table.  
Cette action s'effectue au moyen de l'ordre [`INSERT INTO`](https://sqlite.org/lang_insert.html) : 

```sql
INSERT INTO auteur 
VALUES (97, 'Ritchie', 'Dennis'), 
    (98, 'Voltaire', ''), 
    (103, 'Toriyama', 'Akira');
```

On spécifie le nom de la table (ici `auteur`) suivi d'une suite de $n$-uplets, chacun entre parenthèses.  
Chaque $n$-uplet représente une nouvelle ligne de la table.  
Les valeurs des attributs sont supposés être dans le même ordre que lors du `CREATE TABLE`.  
Si on souhaite les passer dans un ordre différent, on peut spécifier l'ordre des attributs avant le mot clé `VALUES`

```sql
INSERT INTO auteur (prenom, a_id, nom) 
VALUES ('Jean-Jacques', 200, 'Rousseau');
```

Un point important est que les contraintes d'intégrité sont vérifiées au moment de l'insertion.  
Une instruction `INSERT` violant ces contraintes conduira donc à une erreur et les données correspondantes ne seront pas ajoutées à la table.

```sql
INSERT INTO auteur 
VALUES (97, 'Dumas', 'Alexandre');
```
```
Result: UNIQUE constraint failed: auteur.a_id
```

Ici, on essaye d'ajouter un auteur avec la même clé primaire qu'un auteur déjà existant.

```sql
INSERT INTO produit 
VALUES (1, 'ordinateur', 10, -200);
```
```
Result: CHECK constraint failed: produit
```

Ici, l'insertion viole la contrainte `CHECK` de la table `produit` définie plus haut.

## SQL et Python
### Présentation
L'une des difficultés d'une telle présentation repose sur le fait que, pour chaque SGBD existant, certaines lignes spécifiques propres à ce SGBD sont nécessaires.
Ainsi, ces lignes seront différentes selon que l'on se connecte à PostgreSQL, MariaDB ou encore Oracle.

Pour interagir avec une base de données au format SQLite3 depuis Python, il est nécessaire d'utiliser le module [`sqlite3`](https://docs.python.org/fr/3/library/sqlite3.html).  
Ensuite, il faut effectuer un certain nombre d'étapes :

* Se connecter à la base de données
* Exécuter la requête
* Éventuellement, exploiter le résultat de la requête
* Si la base a été modifiée, valider les changements
* Se déconnecter de la base

Un aspect important du langage Python est que ces concepteurs ont défini une interface unifiée d'accès aux bases de données.  
Ainsi, même si les SGBD visés sont différents, les méthodes Python utilisées seront toujours les mêmes, ce qui rend le code facilement portable d'un SGBD à un autre.  
Nous avons choisi d'importer le module `sqlite3` sous le nom générique `sgbd` ce qui évitera de devoir changer de nom dans la suite du programme si on change de SGBD.

### Connexion et déconnexion
Pour une base SQLite3, ces 2 étapes sont très simples :

#### Connexion
On commence par ouvrir une connexion avec la base de données, en appelant la méthode  [`connect`](https://docs.python.org/fr/3/library/sqlite3.html#sqlite3.connect) :

```python
connexion = sgbd.connect("NomFichierBase.db")
```

La base sera en effet stockée dans un fichier dont le nom est suffixé par `.db`.
#### Déconnexion
On ferme ensuite la connexion, en appelant la méthode [`close`](https://docs.python.org/fr/3/library/sqlite3.html#sqlite3.Connection.close) :

```python
connexion.close()
```

### Valider les changements
Les bases de données sont des systèmes **transactionnels**.  
Cela signifie qu'un programme qui interagit avec une base de données fait ses modifications en mémoire, jusqu'à ce que celles-ci soient validées.  
Lors de cette validation, si les modifications sont toujours possibles (ce n'est pas forcément le cas, si un autre utilisateur a modifié la base entre temps), elles sont toutes transférées dans la base de données sur le disque. Sinon, aucune n'est effectuée.  
C'est cette validation par paquet qui constitue une transaction. 

En Python, les transactions sont validées en appelant la méthode [`commit`](https://docs.python.org/fr/3/library/sqlite3.html#sqlite3.Connection.commit) sur l'objet `connexion`.  
Chaque `commit` termine la transaction précédente, puis en commence une autre.  
La première transaction est initiée lors de la connexion à la base de données.  
Si aucune modification n'a été apportée à la base, la phase de validation n'est pas nécessaire.  
Dans le cas contraire, on valide nos modifications ainsi :

```python
connexion.commit()
```

### Exécuter une requête 
Pour exécuter une requête, il faut la passer en paramètre de la méthode [`execute`](https://docs.python.org/fr/3/library/sqlite3.html#sqlite3.Connection.execute) de l'objet `connexion`.

#### Création d'une table
Voici, par exemple, le code d'un programme Python permettant de créer la table `auteur` :

In [ ]:
import sqlite3 as sgbd

#connexion à la base
connexion = sgbd.connect("mediatheque.db")
#suppression éventuelle de l'ancienne table
connexion.execute("DROP TABLE IF EXISTS auteur")
#creation de la table Editeur
connexion.execute("""CREATE TABLE auteur (
                            a_id INT PRIMARY KEY,
                            nom VARCHAR(90) NOT NULL,
                            prenom VARCHAR(90) NOT NULL
                          )""")
#validation
connexion.commit()
#déconnexion
connexion.close()

#### Ajout de données dans une table
Pour ajouter des données dans une table, on peut utiliser la méthode `execute` présentée précédemment :

In [ ]:
import sqlite3 as sgbd

connexion = sgbd.connect("mediatheque.db")
connexion.execute("""INSERT INTO auteur 
                        VALUES (97, 'Ritchie', 'Dennis')""")
connexion.commit()
connexion.close()

Lorsqu'on désire insérer plusieurs données dans une table, on peut utiliser avantageusement la méthode [`executemany`](https://docs.python.org/fr/3/library/sqlite3.html#sqlite3.Connection.executemany).  
Celle-ci prend 2 paramètres : 
* une requête paramétrée (des `?` signalent les zones paramétrées) 
* une liste de tuples 
la requête est exécutée pour chaque tuple de la liste, en remplaçant le $i$-ème point d'interrogation par la $i$-ème composante du tuple.  

Par exemple :

In [ ]:
import sqlite3 as sgbd

connexion = sgbd.connect("mediatheque.db")
liste_auteurs = [
    (103, 'Toriyama', 'Akira'),
    (200, 'Rousseau', 'Jean-Jacques')
    ]
connexion.executemany("INSERT INTO auteur VALUES(?, ?, ?)", liste_auteurs)
connexion.commit()
connexion.close()

## Exercices
### Exercice 1
Regrouper ensemble les termes synonymes:  

    colonne, entité, domaine, attribut, ligne, schéma, base de données, type, column, row

### Exercice 2
On considère la modélisation :

`Annuaire`(`nom` String, `prénom` String, <u><code>tel</code> String</u>)

Donner un ordre SQL permettant de créer la table correspondante, avec un maximum de contrainte d'intégrité.

### Exercice 3
On considère la modélisation :

`Eleve`(`nom` String, `prénom` String, <u><code>num</code> String</u>)

`Matiere`(`intitule` String, <u><code>m_id</code> INT</u>)

`Note`(<u><code># num</code> String, <code># m_id</code> INT</u>, `note` Float).  
Les attributs `num` et `m_id` sont des clés étrangères faisant référence aux attributs `num` de la relation `Auteur` et `m_id` de la relation `Livre` respectivement.

Donner les ordres SQL permettant de créer les tables correspondantes, avec un maximum de contrainte d'intégrité. 

Donner les ordres SQL permettant de supprimer ces tables une fois qu'elles existent.

### Exercice 4
Pour chacune des séquences d'ordres SQL suivantes, dire quelle instruction provoque une erreur.  
On suppose que la base de données ne contient aucune table au début de chaque séquence.

1 .
```sql
DROP TABLE client;
CREATE TABLE client (cid INT PRIMARY KEY, nom VARCHAR(100), 
                     prenom VARCHAR(100), points_fidelite INT NOT NULL, 
                     CHECK (points_fidelite >= 0));
```

2 .
```sql
CREATE TABLE client (cid INT PRIMARY KEY, nom VARCHAR(100),
                     prenom VARCHAR(100), points_fidelite INT NOT NULL, 
                     CHECK (points_fidelite >= 0));                     
CREATE TABLE commande (cid INT REFERENCES client(cid),
                       pid INT REFERENCES produit(pid), 
                       date DATE NOT NULL);                  
CREATE TABLE produit (pid INT PRIMARY KEY,nom VARCHAR(100), prix DECIMAL(10,2));
``` 

3 . 
```sql
CREATE TABLE client (cid INT PRIMARY KEY, nom VARCHAR(100), 
                     prenom VARCHAR(100), points_fidelite INT NOT NULL, 
                     CHECK (points_fidelite >= 0));
CREATE TABLE produit (pid INT PRIMARY KEY, nom VARCHAR(100), prix DECIMAL(10,2));
CREATE TABLE commande (cid INT REFERENCES client(cid), 
                       nomp VARCHAR(100) REFERENCES produit (nom),
                       date DATE NOT NULL);
```

4 .
```sql
CREATE TABLE client (cid INT PRIMARY KEY, nom VARCHAR(100), 
                     prenom VARCHAR(100), points_fidelite INT NOT NULL,
                     CHECK (points_fidelite >= 0));
CREATE TABLE produit (pid INT PRIMARY KEY, nom VARCHAR(100), prix DECIMAL(10,2));
CREATE TABLE commande (cid INT REFERENCES client(cid),
                       pid INT REFERENCES produit(pid),
                       date DATE NOT NULL);
INSERT INTO commande VALUES (0, 0, '2020-03-02');
```

### Exercice 5
On considère les deux tables suivantes:
```sql
CREATE TABLE joueur (jid INT PRIMARY KEY, nom VARCHAR(100) NOT NULL);

CREATE TABLE partie (j1 INT REFERENCES joueur(jid), j2 INT REFERENCES joueur(jid),
                     score1 INT NOT NULL, score2 INT NOT NULL, 
                     CHECK(j1 <> j2));
```                  
Ces tables stockent des résultats de parties entre des joueurs.  

Lister toutes les contraintes d'intégrité et pour chacune donner des ordres SQL violant ces contraintes.

### Exercice 6
Modifier les ordres de création de table de l'exercice précédent pour prendre en compte les modifications suivantes:
* La table partie contient en plus une colonne jour non nulle, indiquant la date à laquelle la partie à eu lieu.
* Les scores ne peuvent pas être négatifs.
* Deux joueurs ne peuvent pas jouer plusieurs fois le même jour.

### Exercice 7
Écrire un programme Python qui lit un fichier CSV `infos.csv` au format suivant:
* les champs sont séparés par des `;`
* le fichier contient 4 colonnes `nom`, `prenom`, `annee_naissance`, `taille`, représentant le nom, prénom, l'année de naissance et la taille (en cm) de personnes.

Le programme doit créer un fichier SQL `infos.sql` qui :
1. crée une table permettant de stocker ces informations ainsi qu'un identifiant unique (entier) servant de clé primaire 
2. remplit la table avec les données du fichier CSV.

## Travaux pratiques
* [Bon voyage Monsieur Dumollet](Travaux_Pratiques/TP_Voyage_Dumollet.ipynb)

## Liens :
* [SQLite](https://www.sqlite.org/index.html)
* [DB Browser for SQLite](https://sqlitebrowser.org/)
* [FLOT/MOOC : Bases de données relationnelles](http://flot.sillages.info/?portfolio=bases-de-donnees-relationnelles)
* Fichiers `odb` (LibreOffice Base) : [Bibliothèque](Fichiers/Bibliotheque.odb)
* Base de données de la médiatèque : [mediatheque.sql](Fichiers/mediatheque.sql)